**Name: Ali Oku <br>
Z-ID: Z1893417 <br>
Assignment 7<br>**

**Download & Extract Files**

In [1]:
import requests
import zipfile
import pandas as pd
import concurrent.futures
from pathlib import Path
import os

In [2]:
if os.path.exists("archive.zip") == False:
    r = requests.get("http://faculty.cs.niu.edu/~dakoop/cs503-2021sp/a7/archive.zip")
    open('archive.zip', 'wb').write(r.content)
        
if os.path.exists("./grocery-data/") == False:
    with zipfile.ZipFile("archive.zip","r") as zip_ref:
        zip_ref.extractall()

**Find Matching Files**

In [3]:
list_of_files = []
p = Path('.')
for name in p.glob('**/*2021*03*csv'):
    list_of_files.append(str(name))

list_of_files

['grocery-data/southeast_2021_03.csv',
 'grocery-data/2021-03-nw.csv',
 'grocery-data/great-lakes-csv/2021_03.csv',
 'grocery-data/alaska/2021/2021-03.csv',
 'grocery-data/ne/2021-03.csv']

**Use Threads to Process Files**

In [4]:
def process_files(file_list):
    list_of_csv = []
    
    for filepath in file_list:
        df = pd.read_csv(filepath)
        df = df[df["Date"] == "2021-03-31"]
        df["Filename"] = filepath
        df.set_index("Date")

        list_of_csv.append(df)
        
    return list_of_csv

with concurrent.futures.ThreadPoolExecutor(max_workers = 5) as executor:
    results = executor.submit(process_files, list_of_files)
    
pd.concat(results.result()).to_csv("2021-03-31-all.csv", index = False)